In [1]:
import numpy as np
import matplotlib.pylab as plt
import math as math
from scipy.optimize import fmin
from iminuit import Minuit, describe, Struct

%matplotlib  notebook

In [79]:
# Add code to the following function so that it takes in two datasets, loops over one of them, and finds
# information about the nearest neighbors in the other dataset, based on a flag. 

def nn(data0,data1,r=None,nneighbors=None):
    
    ret = -1
    ret_list=[]
    if r is not None and nneighbors is not None:
        exit(-1)
        return ret
    elif r is not None and nneighbors is None:
        for num0 in data0:
            count=0
            diff = np.abs(num0 - data1)
            count = len(diff[diff<r])
            ret_list.append(float(count)/(float(len(data1))*r))
        ret_list = np.array(ret_list)
        return ret_list
    elif r is None and nneighbors is not None:
        for num0 in data0:
            diff = np.abs(num0 - data1)
            diff.sort()
            radius= diff[nneighbors-1]
            ret_list.append(1/radius)
        ret_list = np.array(ret_list)
        return ret_list
    return ret

In [87]:
def normal(x,mean,width):
    return (1.0/(width*np.sqrt(2*np.pi)))*(np.exp(-(x-mean)**2/(2*(width**2))))

def calc_pull(iterations, dataPoints, MC, rad):
    pull_frac_list=[]
    average_best_frac = 0
    frac = []
    fit_frac = []
    fit_frac_uncert = []
    
    

    for num in range(iterations):
        #Nsig = dataPoints
        Nsig = np.random.poisson(dataPoints)
        sig_mean=10.4
        sig_width=.06
        signal = np.random.normal(sig_mean,sig_width,Nsig)
        Nbkg = 1000
        #Nbkg = np.random.poisson(Nbkg)
        background = 9.0+(2*np.random.random(Nbkg))
        data = signal.copy()
        data = np.append(data,background.copy())

        frac.append(float(Nsig)/(float(Nbkg)+float(Nsig)))
        Nsig_compare = MC
        sig_mean_compare=10.4
        sig_width_compare = .06
        signal_compare = np.random.normal(sig_mean_compare,sig_width_compare,Nsig_compare)
        
        Nbkg_compare= MC
        background_compare= 9.0+(2*np.random.random(Nbkg_compare))
        
        def probability(frac):
            tot_prob=[]
            #signal_prob=nn(data,signal_compare, nneighbors=100)
            #background_prob= nn(data, background_compare, nneighbors=100)
            #signal_prob=nn(data,signal_compare, r=rad)
            #background_prob= nn(data, background_compare, r=rad)
            signal_prob = normal(data,10.4,0.06)
            background_prob = 0.5*np.ones(len(data))
            tot_prob.append(frac*signal_prob+ ((1-frac)*background_prob))
            tot_prob=np.array(tot_prob)
            tot_prob[tot_prob<=0.0] = 1e-64
            #print -np.log(tot_prob[tot_prob>0]).sum() - -np.log(tot_prob).sum()
            #return -np.log(tot_prob[tot_prob>0]).sum()
            return -np.log(tot_prob).sum()
        
        m=Minuit(probability, frac= 0.25, limit_frac=(0.001,1), error_frac=0.001,  errordef = 0.5, print_level=0)
        m.migrad()
        #m.hesse()
        if (m.get_fmin().is_valid):
            param=m.values
            err=m.errors
            fit_frac.append(param["frac"])
            fit_frac_uncert.append(err["frac"])
            print Nsig,Nbkg,(float(Nsig)/(float(Nbkg)+float(Nsig))),param["frac"],err["frac"]
            pull_frac=((float(Nsig)/(float(Nbkg)+float(Nsig)))-param["frac"])/err["frac"]
            ndata = len(data)
            frac_org = Nsig/float(Nsig + Nbkg)
            nsig_org = frac_org * ndata
            nsig_fit = param["frac"]
            nsig_err = err["frac"]
            pull_frac_list.append(pull_frac)
    return pull_frac_list, frac, fit_frac, fit_frac_uncert,iterations

In [88]:
means=[]
stds=[]

for sig in [500]:
    for MC in [5000]:
        for rad in [0.01]: #, 0.10,0.20,0.50]:
            
            print 'number of signal points: %s \t number of MC point: %s \tradius: %s' % (sig,MC,rad)
            pulls,org_vals,fit_vals,fit_uncerts,iteration = calc_pull(20, sig, MC, rad)
            #plt.figure()
            #plt.hist(pulls, bins=50)
            #x = np.linspace(-5,5,1000)
            #y=normal(x,0,1)
            #plt.plot(x,(iteration*7/(50))*y,'y-',linewidth=2)
            parameters_mean = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean pulls': np.mean(pulls)}
            means.append(parameters_mean)
            parameters_std = {'signal': sig, 'MC_points': MC, 'radius': rad, 'mean stds': np.std(pulls)}
            stds.append(parameters_std)
            print "-----------------"
            print pulls
            

#plt.figure(figsize=(6,6))
point_color='bo'
sizes=0


print "---------------"
print means
print stds

#WITH THE POISSON and 100 iterations- TESTING IT

number of signal points: 500 	 number of MC point: 5000 	radius: 0.01
492 1000 0.329758713137 0.325018404739 0.0147285063766
537 1000 0.349381912817 0.345687805095 0.0145892946591
536 1000 0.348958333333 0.357078920313 0.0147526539915
490 1000 0.328859060403 0.327701492472 0.0147762997629
466 1000 0.317871759891 0.325426367706 0.0150018997963
522 1000 0.34296977661 0.345022363761 0.0147049535388
537 1000 0.349381912817 0.349542326208 0.0147233877513
519 1000 0.341672152732 0.337683847641 0.0146402075515
510 1000 0.337748344371 0.330798617394 0.0146439590244
487 1000 0.327505043712 0.326683362404 0.0147856828686
517 1000 0.340804218853 0.338706035932 0.0147554995241
529 1000 0.345977763244 0.349706203551 0.0146446811494
495 1000 0.33110367893 0.335406586507 0.014714085726
471 1000 0.320190346703 0.318199490622 0.0148169674413
488 1000 0.327956989247 0.307718696027 0.014637758122
517 1000 0.340804218853 0.33230926482 0.0147467852127
441 1000 0.306037473976 0.302256638179 0.0148394349704


In [78]:
print means
print stds


[{'signal': 50, 'radius': 0.01, 'mean pulls': -0.077027357838128349, 'MC_points': 5000}]
[{'signal': 50, 'radius': 0.01, 'mean stds': 0.91276470882102401, 'MC_points': 5000}]
